In [242]:
import os
import pandas as pd
from dotenv import load_dotenv
from py2neo import Graph

load_dotenv()
password = os.getenv('DBPASS')

graph = Graph("bolt://localhost:7687", auth=("neo4j", password))

In [243]:
# not really working yet
#query = """
#MATCH (i:Issuelist {name: "Umwelt"})<-[:ABOUT]-(b:Bill:Motion)<-[:SPONSORS]-(p:Person)-[:GAVE]->(s:Speech)
#RETURN DISTINCT p, s;
#"""

#speeches1 = graph.run(query).data()
#len(speeches1)

In [244]:
#speeches1[1]

In [245]:
#query = """
#MATCH (i:Issuelist {name: "Umwelt"})<-[:ABOUT]-(b:Bill:Motion)<-[:SPONSORS]-(p:Person)-[:GAVE]->(s:Speech)
#RETURN DISTINCT b, p, s;
#"""

#speeches2 = graph.run(query).data()
#len(speeches2)

In [246]:
#len(speeches2)/len(speeches1)

In [247]:
# number of bills per year and canton
query_bills_cantons = """
MATCH (i:Issuelist)<-[:ABOUT]-(b:Bill)<-[:SPONSORS]-(p:Person)-[:REPRESENTS]->(c:Canton), 
    (p)-[:ELECTED_TO]->(ch:Chamber),
    (b)-[:CONTAINS]->(d:`Chamber Debate`)
    
RETURN i.name AS issue, 
    c.name AS canton, 
    ch.name AS chamber,
    COUNT(DISTINCT b) AS num_bills, 
    substring(toString(d.date_end), 0, 4) AS year
    
ORDER BY issue, canton, year;
"""

results_bills_cantons = graph.run(query_bills_cantons).data()
df_bills_cantons = pd.DataFrame(results_bills_cantons)

# Display the DataFrame as a table
df_bills_cantons

,issue,canton,chamber,num_bills,year
0,Beschäftigung und Arbeit,Aargau,Nationalrat,2,2012
1,Beschäftigung und Arbeit,Aargau,Ständerat,1,2012
2,Beschäftigung und Arbeit,Aargau,Nationalrat,2,2013
3,Beschäftigung und Arbeit,Aargau,Nationalrat,6,2014
4,Beschäftigung und Arbeit,Aargau,Ständerat,1,2014
...,...,...,...,...,...
12207,Zivilrecht,Zürich,Nationalrat,15,2021
12208,Zivilrecht,Zürich,Nationalrat,19,2022
12209,Zivilrecht,Zürich,Ständerat,2,2022
12210,Zivilrecht,Zürich,Ständerat,1,2023


In [248]:
df_bills_cantons[(df_bills_cantons['year'] == '2012') & (df_bills_cantons['canton'] == 'Aargau') & (df_bills_cantons['issue'] == 'Energie')]



,issue,canton,chamber,num_bills,year
812,Energie,Aargau,Nationalrat,15,2012
813,Energie,Aargau,Ständerat,1,2012


In [253]:
# query including political parties
query_votes_canton_group = """
MATCH (i:Issuelist)<-[:ABOUT]-(b:Bill)<-[:SPONSORS]-(p:Person)-[:REPRESENTS]->(c:Canton), 
    (pg:`Parliamentary Group`)<-[:MEMBER_OF]-(p),
    (p)-[:ELECTED_TO]->(ch:Chamber),
    (b)-[:CONTAINS]->(d:`Chamber Debate`)
    
RETURN i.name AS issue, 
    substring(toString(d.date_end), 0, 4) AS year,
    labels(b)[1] AS bill_type,
    c.name AS canton, 
    ch.name AS chamber,
    pg.name AS parl_group,
    COUNT(DISTINCT b) AS num_bills
    
ORDER BY issue, canton, year
LIMIT 10;
"""

results_votes_canton_group = graph.run(query_votes_canton_group).data()
df_votes_canton_group = pd.DataFrame(results_votes_canton_group)

# Display the DataFrame as a table
df_votes_canton_group

,issue,year,bill_type,canton,chamber,parl_group,num_bills
0,Beschäftigung und Arbeit,2012,Question Time,Aargau,Nationalrat,Fraktion der Schweizerischen Volkspartei,1
1,Beschäftigung und Arbeit,2012,Question Time,Aargau,Nationalrat,Sozialdemokratische Fraktion,1
2,Beschäftigung und Arbeit,2012,Question Time,Aargau,Ständerat,Fraktion der Schweizerischen Volkspartei,1
3,Beschäftigung und Arbeit,2013,Question Time,Aargau,Nationalrat,Sozialdemokratische Fraktion,2
4,Beschäftigung und Arbeit,2014,Interpellation,Aargau,Ständerat,Freisinnig-demokratische Fraktion,1
5,Beschäftigung und Arbeit,2014,Interpellation,Aargau,Ständerat,FDP-Liberale Fraktion,1
6,Beschäftigung und Arbeit,2014,Motion,Aargau,Nationalrat,Fraktion BD,1
7,Beschäftigung und Arbeit,2014,Interpellation,Aargau,Nationalrat,Sozialdemokratische Fraktion,2
8,Beschäftigung und Arbeit,2014,Motion,Aargau,Nationalrat,Sozialdemokratische Fraktion,1
9,Beschäftigung und Arbeit,2014,Interpellation,Aargau,Nationalrat,FDP-Liberale Fraktion,1


In [239]:
df_votes_canton_group[(df_votes_canton_group['year'] == '2012') & (df_votes_canton_group['canton'] == 'Aargau') & (df_votes_canton_group['issue'] == 'Energie')]



,issue,year,canton,chamber,parl_group,num_bills
2372,Energie,2012,Aargau,Nationalrat,Fraktion der Schweizerischen Volkspartei,1
2373,Energie,2012,Aargau,Ständerat,Sozialdemokratische Fraktion,1
2374,Energie,2012,Aargau,Nationalrat,Sozialdemokratische Fraktion,4
2375,Energie,2012,Aargau,Nationalrat,Grüne Fraktion,9
2376,Energie,2012,Aargau,Nationalrat,Fraktion BD,1
